## Import the Library

In [1]:
from bs4 import BeautifulSoup
import requests

#### Download the web page containing the forecast.

In [2]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")

#### Create a BeautifulSoup class to parse the page.

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

#### Find the div with id seven-day-forecast, and assign to seven_day

In [4]:
seven_day = soup.find(id="seven-day-forecast")

#### Inside seven_day, find each individual forecast item.

In [5]:
forecast_items = seven_day.find_all(class_="tombstone-container")

#### Extract and print the first forecast item.

In [6]:
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Today
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. "/>
 </p>
 <p class="short-desc">
  Sunny
 </p>
 <p class="temp temp-high">
  High: 69 °F
 </p>
</div>


#### We'll extract the name of the forecast item

In [7]:
period = tonight.find(class_ = "period-name").get_text()
short_Desc = tonight.find(class_ = "short-desc").get_text()
temp = tonight.find(class_ = "temp temp-high").get_text()

print(period)
print(short_Desc)
print(temp)

Today
Sunny
High: 69 °F


#### we can extract the title attribute from the img tag

In [8]:
img = tonight.find('img')

In [9]:
img.prettify()

'<img alt="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. " class="forecast-icon" src="newimages/medium/few.png" title="Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. "/>\n'

In [10]:
desc =img['title']
print(desc)

Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. 


#### I use list comprehension to call the get_text method on each BeautifulSoup object

In [11]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Today',
 'Tonight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight',
 'Friday']

In [12]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Sunny']
['High: 69 °F', 'Low: 55 °F', 'High: 67 °F', 'Low: 54 °F', 'High: 68 °F', 'Low: 55 °F', 'High: 68 °F', 'Low: 56 °F', 'High: 70 °F']
['Today: Sunny, with a high near 69. West wind 11 to 18 mph, with gusts as high as 23 mph. ', 'Tonight: Partly cloudy, with a low around 55. West southwest wind 15 to 21 mph, with gusts as high as 25 mph. ', 'Tuesday: Mostly sunny, with a high near 67. West southwest wind 11 to 20 mph, with gusts as high as 24 mph. ', 'Tuesday Night: Partly cloudy, with a low around 54. West southwest wind 16 to 18 mph, with gusts as high as 23 mph. ', 'Wednesday: Mostly sunny, with a high near 68. West southwest wind 7 to 17 mph, with gusts as high as 21 mph. ', 'Wednesday Night: Partly cloudy, with a low around 55.', 'Thursday: Mostly sunny, with a high near 68.', 'Thursday Night: Partly cloudy, with a low around 56.', 'Friday: Mos

#### Combining our data into a Pandas Dataframe

In [13]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,period,short_desc,temp,desc
0,Today,Sunny,High: 69 °F,"Today: Sunny, with a high near 69. West wind 1..."
1,Tonight,Partly Cloudy,Low: 55 °F,"Tonight: Partly cloudy, with a low around 55. ..."
2,Tuesday,Mostly Sunny,High: 67 °F,"Tuesday: Mostly sunny, with a high near 67. We..."
3,TuesdayNight,Partly Cloudy,Low: 54 °F,"Tuesday Night: Partly cloudy, with a low aroun..."
4,Wednesday,Mostly Sunny,High: 68 °F,"Wednesday: Mostly sunny, with a high near 68. ..."
5,WednesdayNight,Partly Cloudy,Low: 55 °F,"Wednesday Night: Partly cloudy, with a low aro..."
6,Thursday,Mostly Sunny,High: 68 °F,"Thursday: Mostly sunny, with a high near 68."
7,ThursdayNight,Partly Cloudy,Low: 56 °F,"Thursday Night: Partly cloudy, with a low arou..."
8,Friday,Mostly Sunny,High: 70 °F,"Friday: Mostly sunny, with a high near 70."


#### Use a regular expression and the Series.str.extract method to pull out the numeric temperature values:

In [14]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    69
1    55
2    67
3    54
4    68
5    55
6    68
7    56
8    70
Name: temp_num, dtype: object

In [15]:
weather["temp_num"].mean()

62.44444444444444

In [16]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0    False
1     True
2    False
3     True
4    False
5     True
6    False
7     True
8    False
Name: temp, dtype: bool

In [17]:
weather[is_night]

,period,short_desc,temp,desc,temp_num,is_night
1,Tonight,Partly Cloudy,Low: 55 °F,"Tonight: Partly cloudy, with a low around 55. ...",55,True
3,TuesdayNight,Partly Cloudy,Low: 54 °F,"Tuesday Night: Partly cloudy, with a low aroun...",54,True
5,WednesdayNight,Partly Cloudy,Low: 55 °F,"Wednesday Night: Partly cloudy, with a low aro...",55,True
7,ThursdayNight,Partly Cloudy,Low: 56 °F,"Thursday Night: Partly cloudy, with a low arou...",56,True


## Thank You. To learn more about it Go to https://themenyouwanttobe.wordpress.com 